In [1]:
# Utility script

# statistic

In [2]:
import numpy as np
import cv2
from math import sqrt, atan2, pi

In [3]:
from util_registration import transf_2stage_estim

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
def outlier_remove(par4_tuple,bound4_tuple):
    '''
    Check four (error) arrays to be within specified boundaries (by absolute value).
    Input:
        par4_tuple - tuple of 4 arrays (of equal length);
        bound4_tuple - tuple of 4 boundaries (scalars).
    Output:
        good_par4__tuple - tuple of 4 arrays with only "good indexes" elements
    		("good indexes" - when for current index all 4 elements within the relevant boundaries);
        indexes - "good indexes".
    '''
    sh_x_bound, sh_y_bound, sc_bound, rot_bound = bound4_tuple
    sh_x, sh_y, sc, rot = par4_tuple
    
    is_good = ( np.abs(sh_x) < sh_x_bound ) * \
              ( np.abs(sh_y) < sh_y_bound ) * \
              ( np.abs(sc)   < sc_bound ) * \
              ( np.abs(rot)  < rot_bound )
    is_good = is_good.astype(bool)
    
    good__sh_x = sh_x[is_good]
    good__sh_y = sh_y[is_good]
    good__sc = sc[is_good]
    good__rot = rot[is_good]
    good_par4__tuple = (good__sh_x, good__sh_y,
                        good__sc, good__rot)
    
    indexes = is_good.nonzero()
    # indexes - tuple with one ndarray 
    
    return good_par4__tuple, indexes[0]

In [5]:
def monteCarlo_reg(ri,ci_clean,
                   gt_true,
                   snr,nTest):   
    '''
    Evaluate registration RMSE for 4 parameters by monte-carlo:
        snr == CONSTANT;
        averaging over ONE fragment (with different noise realizations).
    
    Inputs:
        ri - referense image;
        ci_clean - current image without noise;
        gt_true - true geometric transformation: (center_x, center_y, scale, rot),
                                    rot in degrees;
        snr -  signal noise ratio (not dB, by standard deviation ratio);
        nTest - number of monte-carlo tests.
    
    Output:
        Stat_ - dict for estimated charasteristics: RMSE (for 4 parameters) and P (correct registration probability),
            RMSE evaluated only for correct registrations (errors within adjusted bounds)
            [the above characteristics evaluated for (1) (with suffix "__kp") and 
                                                     (2) (with suffix "__ncc") steps];
        err_kp - errors from 1st step (feature-based registration);
        err_ncc - errors from 2nd step [ (1) feature-based registration + (2) NCC optimization].
        
    Comments:
        ci_clean must be squared.              # XXX - Attension.
        ri must be bigger than ci_clean.  
    '''
    
    
    # Bounds on errors.
    sh_b = 2
    sc_b = 0.05
    rot_b = 5
    bound4_tuple = (sh_b, sh_b, sc_b, rot_b)
    
    ci_s = ci_clean.shape[0]
    
    
    #%% Monte-Carlo ----------------
    err_kp = np.zeros( (nTest,4) )
    err_ncc = np.zeros( (nTest,4) )
    
    ci_E = np.std(ci_clean)
    
    
    # Compute kp (keypoint coordinates) and des (descriptors) for RI (evaluated once for all iterations).
    #    XXX - The choise of detector (and descriptor) here should be match with 
    #          corresponding item in function keypoint_estim.
    kp_type = 'sift'
    if kp_type == 'sift':
        # kp_obj = cv2.xfeatures2d.SIFT_create()
        kp_obj = cv2.SIFT_create()
    elif kp_type == 'kaze':
        kp_obj = cv2.KAZE_create()
    elif kp_type == 'akaze':
        kp_obj = cv2.AKAZE_create()
    else:
        raise ValueError('Wrong value for kp_obj')
    
    
    
    ri_scaled = cv2.normalize(ri, None, alpha=0, beta=255, 
                              norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    ri_kp, ri_des = kp_obj.detectAndCompute(ri_scaled, None) 
    ri_kp_des = (ri_kp, ri_des)
    

    for k in range(nTest):
        noise_SD = ci_E/snr
        ci = ci_clean + np.random.normal(0,noise_SD, (ci_s,ci_s) )
        
        # Estimation (without bootstrap).
        # (gt_kp, gt_ncc) = transf_2stage_estim(ri,ci,ri_kp_des = ri_kp_des)
        #
        # XXX - speed-up mode (when true_gt_tuple is defined): 
        #       if 1st registration step estimates bad - we don't evaluate 2nd step;
        #       (in real situation we don't know how bad or not 1st step estimates)
        (gt_kp, gt_ncc) = transf_2stage_estim(ri,ci,ri_kp_des = ri_kp_des,
                                              true_gt_tuple = gt_true)  
        
        
        err_kp[k,:] = np.array(gt_kp) - np.array(gt_true)
        err_ncc[k,:] = np.array(gt_ncc) - np.array(gt_true)
    #----------------------------------------
    
    
    
    
    #%% Use criterion for corect registration (to select correct registrations errors).
    kp_err4_tuple = (err_kp[:,0],
                     err_kp[:,1],
                     err_kp[:,2],
                     err_kp[:,3])
    (good_kp_err4_tuple,
     _)                = outlier_remove(kp_err4_tuple,
                                        bound4_tuple)
                                        
    ncc_err4_tuple = (err_ncc[:,0],
                      err_ncc[:,1],
                      err_ncc[:,2],
                      err_ncc[:,3])                                
    (good_ncc_err4_tuple,
     _)                = outlier_remove(ncc_err4_tuple,
                                        bound4_tuple)                                


    
    (err_x__kp, err_y__kp, err_sc__kp, err_rot__kp) = good_kp_err4_tuple
    (err_x__ncc, err_y__ncc, err_sc__ncc, err_rot__ncc) = good_ncc_err4_tuple
    

    #%%
    Stat_ = {}
    Stat_['rmse_x__kp'] = np.sqrt( np.mean(err_x__kp**2) )
    Stat_['rmse_y__kp'] = np.sqrt( np.mean(err_y__kp**2) )
    Stat_['rmse_sc__kp'] = np.sqrt( np.mean(err_sc__kp**2) )
    Stat_['rmse_rot__kp'] = np.sqrt( np.mean(err_rot__kp**2) )
    
    Stat_['rmse_x__ncc'] = np.sqrt( np.mean(err_x__ncc**2) )
    Stat_['rmse_y__ncc'] = np.sqrt( np.mean(err_y__ncc**2) )
    Stat_['rmse_sc__ncc'] = np.sqrt( np.mean(err_sc__ncc**2) )
    Stat_['rmse_rot__ncc'] = np.sqrt( np.mean(err_rot__ncc**2) )
    
    Stat_['P__kp'] = np.size(err_x__kp) / nTest
    Stat_['P__ncc'] = np.size(err_x__ncc) / nTest
    
    return (Stat_, err_kp, err_ncc)